In [1]:
# Needed to switch directory in Google drive so as to import MARL env.
from google.colab import drive 
drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/"
!pwd
!pip install -r requirements.txt
!pip show tensorflow
!pip show ray

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction
/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction
     |████████████████████████████████| 1.2MB 5.0MB/s 
     |████████████████████████████████| 112kB 59.2MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 829kB 52.3MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 2.7MB 37

Name: tensorflow
Version: 2.1.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wrapt, google-pasta, grpcio, wheel, tensorflow-estimator, gast, absl-py, numpy, termcolor, protobuf, six, astor, opt-einsum, scipy, keras-preprocessing, keras-applications, tensorboard
Required-by: tensorflow-federated, stable-baselines, magenta, fancyimpute
Name: ray
Version: 0.8.2
Summary: A system for parallel and distributed Python that unifies the ML ecosystem.
Home-page: https://github.com/ray-project/ray
Author: Ray Team
Author-email: ray-dev@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: filelock, pytest, protobuf, funcsigs, click, py-spy, packaging, pyyaml, aiohttp, grpcio, google, redis, colorama, six, cloudpickle, jsonschema, numpy
Required-by: 

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['RAY_DEBUG_DISABLE_MEMORY_MONITOR'] = "True"

import argparse
import gym
import random
import numpy as np

import ray
from ray import tune
from ray.rllib.utils import try_import_tf
from ray.tune.registry import register_env
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet_v2 import FullyConnectedNetwork
from ray.rllib.models import Model, ModelCatalog
from ray.rllib.policy.policy import Policy
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy

import sys
if "../" not in sys.path:
    sys.path.append("../")

from gym_continuousDoubleAuction.envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv

tf = try_import_tf()

In [0]:
class CustomModel_1(Model):
    def _lstm(self, Inputs, cell_size):
        s = tf.expand_dims(Inputs, axis=1, name='time_major')  # [time_step, feature] => [time_step, batch, feature]
        lstm_cell = tf.nn.rnn_cell.LSTMCell(cell_size)
        self.init_state = lstm_cell.zero_state(batch_size=1, dtype=tf.float32)
        # time_major means [time_step, batch, feature] while batch major means [batch, time_step, feature]
        outputs, self.final_state = tf.nn.dynamic_rnn(cell=lstm_cell, inputs=s, initial_state=self.init_state, time_major=True)
        lstm_out = tf.reshape(outputs, [-1, cell_size], name='flatten_rnn_outputs')  # joined state representation
        return lstm_out

    def _build_layers_v2(self, input_dict, num_outputs, options):
        hidden = 512
        cell_size = 256
        S = tf.layers.flatten(input_dict["obs"]) # input_dict["obs"] is the state
        with tf.variable_scope(tf.VariableScope(tf.AUTO_REUSE, "shared"),
                               reuse=tf.AUTO_REUSE,
                               auxiliary_name_scope=False):
            last_layer = tf.layers.dense(S, hidden, activation=tf.nn.relu, name="fc1")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc2")
        last_layer = tf.layers.dense(last_layer, hidden, activation=tf.nn.relu, name="fc3")

        last_layer = self._lstm(last_layer, cell_size)

        output = tf.layers.dense(last_layer, num_outputs, activation=tf.nn.softmax, name="mu")

        return output, last_layer

In [0]:
def make_RandomPolicy(_seed):

    class RandomPolicy(Policy):
        """
        Hand-coded policy that returns random actions in the env 
        (doesn't learn).
        """

        def __init__(self, observation_space, action_space, config):
            self.observation_space = observation_space
            self.action_space = action_space
            self.action_space.seed(_seed)

        def compute_actions(self,
                            obs_batch,
                            state_batches,
                            prev_action_batch=None,
                            prev_reward_batch=None,
                            info_batch=None,
                            episodes=None,
                            **kwargs):
            """Compute actions on a batch of observations."""
            return [self.action_space.sample() for _ in obs_batch], [], {}

        def learn_on_batch(self, samples):
            """No learning."""
            #return {}
            pass

        def get_weights(self):
            pass

        def set_weights(self, weights):
            pass

    return RandomPolicy

In [5]:
ray.init(ignore_reinit_error=True, log_to_driver=False, webui_host='127.0.0.1', num_cpus=2)

2020-03-18 09:26:30,811	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-18 09:26:30,814	INFO resource_spec.py:212 -- Starting Ray with 6.74 GiB memory available for workers and up to 3.38 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-18 09:26:31,278	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2020-03-18_09-26-30_810775_124/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-18_09-26-30_810775_124/sockets/raylet',
 'redis_address': '172.28.0.2:29445',
 'session_dir': '/tmp/ray/session_2020-03-18_09-26-30_810775_124',
 'webui_url': '127.0.0.1:8265'}

In [0]:
num_agents = 4
num_policies = num_agents
num_iters = 3
simple = False #store_true
num_of_traders = num_agents
tape_display_length = 10
tick_size = 1
init_cash = 1000000
max_step = 700 # per episode
episode = 5 

In [7]:
single_CDA_env = continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step)
obs_space = single_CDA_env.observation_space
act_space = single_CDA_env.action_space
register_env("continuousDoubleAuction-v0", lambda _: continuousDoubleAuctionEnv(num_of_traders, init_cash, tick_size, tape_display_length, max_step))
ModelCatalog.register_custom_model("model_disc", CustomModel_1)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
# Each policy can have a different configuration (including custom model)
def gen_policy(i):
    config = {"model": {"custom_model": "model_disc"},
              "gamma": 0.99,}
    return (None, obs_space, act_space, config)

In [0]:
def policy_mapper(agent_id):
    for i in range(num_agents):
        if agent_id == i:
            return "policy_{}".format(i)

In [0]:
# Dictionary of policies
policies = {"policy_{}".format(i): gen_policy(i) for i in range(num_policies)}

In [0]:
# override policy with random policy

def set_RandomPolicy(policies):
    """
    Set 1st policy as PPO & override all other policies as RandomPolicy with
    different seed.
    """

    for i in range(num_agents):
        if i == num_agents-1:
            break
        x = i + 1
        policies["policy_{}".format(num_policies-x)] = (make_RandomPolicy(num_policies-x), obs_space, act_space, {})

    print('policies:', policies)
    return 0

In [12]:
set_RandomPolicy(policies)

policy_ids = list(policies.keys())

policies: {'policy_0': (None, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {'model': {'custom_model': 'model_disc'}, 'gamma': 0.99}), 'policy_1': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {}), 'policy_2': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {}), 'policy_3': (<class '__main__.make_RandomPolicy.<locals>.RandomPolicy'>, Box(4, 10), Tuple(Discrete(3), Discrete(4), Box(1,), Box(1,), Discrete(12)), {})}


In [13]:
analysis_obj = tune.run(PPOTrainer,
                        local_dir="/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/chkpt/",
                        checkpoint_freq=10, # iterations
                        checkpoint_at_end=True,
                        max_failures=5,
                        resume=False,
                        #restore='/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/chkpt/PPO/PPO_continuousDoubleAuction-v0_2d1f5a70_0_2020-03-17_16-27-21o402ocja/checkpoint_28/checkpoint-28',
                        stop={"timesteps_total": max_step * episode},
                        #stop={"timesteps_total": max_step * episode * 2},
                        config={"env": "continuousDoubleAuction-v0",                     
                        # Number of rollout worker actors to create for parallel sampling.
                        # Setting to 0 will force rollouts to be done in the trainer actor.
                        "num_workers": 1, # Colab (only 2 CPUs or 1 GPU)
                        "num_envs_per_worker": 2,
                        "sample_batch_size": 32, # number of environment steps sampled from each environment
                        "train_batch_size": 128, # minibatch size must be >= 128, number of environment steps sampled from all available environments
                        "multiagent": {"policies_to_train": ["policy_0"],
                                       "policies": policies,
                                       "policy_mapping_fn": policy_mapper,
                                      },
                              },
                        )

Trial name,status,loc
PPO_continuousDoubleAuction-v0_8e49eea8,RUNNING,


Result for PPO_continuousDoubleAuction-v0_8e49eea8:
  custom_metrics: {}
  date: 2020-03-18_09-27-16
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: a1a6cb68061649539c6b88410a8c5ee3
  experiment_tag: '0'
  hostname: 171ebb90f431
  info:
    grad_time_ms: 18208.915
    learner:
      policy_0:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 7.904226303100586
        entropy_coeff: 0.0
        kl: 0.010916304774582386
        policy_loss: -0.13474249839782715
        total_loss: 43314776.0
        vf_explained_var: 0.0
        vf_loss: 43314776.0
    load_time_ms: 157.809
    num_steps_sampled: 128
    num_steps_trained: 128
    sample_time_ms: 3493.013
    update_time_ms: 2360.7
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_perce

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_8e49eea8,RUNNING,172.28.0.2:483,nan,24.4064,128,1


Result for PPO_continuousDoubleAuction-v0_8e49eea8:
  custom_metrics: {}
  date: 2020-03-18_09-27-36
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: a1a6cb68061649539c6b88410a8c5ee3
  experiment_tag: '0'
  hostname: 171ebb90f431
  info:
    grad_time_ms: 17519.119
    learner:
      policy_0:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 7.927767276763916
        entropy_coeff: 0.0
        kl: 0.01411215029656887
        policy_loss: -0.09067018330097198
        total_loss: 1233931.75
        vf_explained_var: 0.0
        vf_loss: 1233931.875
    load_time_ms: 79.484
    num_steps_sampled: 256
    num_steps_trained: 256
    sample_time_ms: 3493.135
    update_time_ms: 1199.433
  iterations_since_restore: 2
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_perc

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_8e49eea8,RUNNING,172.28.0.2:483,nan,44.7725,256,2


...

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_8e49eea8,RUNNING,172.28.0.2:483,0,560.701,3456,27


Result for PPO_continuousDoubleAuction-v0_8e49eea8:
  custom_metrics: {}
  date: 2020-03-18_09-36-34
  done: true
  episode_len_mean: 701.0
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 0
  episodes_total: 4
  experiment_id: a1a6cb68061649539c6b88410a8c5ee3
  experiment_tag: '0'
  hostname: 171ebb90f431
  info:
    grad_time_ms: 16927.445
    learner:
      policy_0:
        cur_kl_coeff: 0.675000011920929
        cur_lr: 4.999999873689376e-05
        entropy: 7.938020706176758
        entropy_coeff: 0.0
        kl: 0.013972057029604912
        policy_loss: -0.060210853815078735
        total_loss: 32535596.0
        vf_explained_var: 0.0
        vf_loss: 32535596.0
    load_time_ms: 1.464
    num_steps_sampled: 3584
    num_steps_trained: 3584
    sample_time_ms: 3776.396
    update_time_ms: 28.529
  iterations_since_restore: 28
  node_ip: 172.28.0.2
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent:

Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_8e49eea8,RUNNING,172.28.0.2:483,0,581.321,3584,28


Trial name,status,loc,reward,total time (s),ts,iter
PPO_continuousDoubleAuction-v0_8e49eea8,TERMINATED,,0,581.321,3584,28


2020-03-18 09:36:35,187	INFO tune.py:352 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.


In [14]:
df = analysis_obj.dataframe()
print(df.T)

                                                                                        0
episode_reward_max                                                                      0
episode_reward_min                                                                      0
episode_reward_mean                                                                     0
episode_len_mean                                                                      701
episodes_this_iter                                                                      0
timesteps_this_iter                                                                   128
done                                                                                 True
timesteps_total                                                                      3584
episodes_total                                                                          4
training_iteration                                                                     28
experiment